In [ ]:
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

from dotenv import load_dotenv
# lädt die (versteckte) Datei die im selben Ordner liegt wie dieses notebook file
# die Datei muss .env benannt sein, diese Dateien sind in der Regel ausgeblendet, 
# weil Dateien mit . andeuten, dass diese im Explorer ausgeblendet werden.
# In der Datei
load_dotenv()

## Header
Wo wir unseren geheimen Schlüssel für ChatGPT mitschicken mit dem wir uns automatisch einloggen und authentifizieren (und auch unser Budget ansprechen)

In [ ]:
import os
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {os.environ["API_KEY"]}",
}

Die Daten die wir mit unsere Anfrage schicken - dies entspricht der GESAMTEN Unterhaltung!

In [ ]:
payload = {
  # Die -mini Modelle sind günstiger per token. Fürs explorieren, macht es Sinn ein günstiges Modell zu benutzen.
  "model": "gpt-4o-mini",
  "messages": [
    {
        "role": "system", 
        "content": "Du bist ein Poet. Du formulierst mir ein Gedicht basierend auf den Nutzereingaben."
    },
    {
      "role": "user",
      # Achtung: Alle informationen die wir hier an OpenAI schicken, werden gespeichert von OpenAI. 
      # Keine sensiblen Informationen oder Details über die eigene/dritte Personen bzw. Informationen mitschicken, 
      # wo man nicht die Rechte dran besitzt.
      "content": "Das Abendbrot war lecker."
    }
  ],
  "temperature": 0.5
}

In [ ]:
import pprint
import requests
ENDPOINT = "https://api.openai.com/v1/chat/completions"
    
response = requests.post(ENDPOINT, headers=headers, json=payload)
if response.status_code != 200:
    logger.warning(f"Fehlerhafte Antwort: {response.text}")

### Rohausgabe der Rückantwort

In [ ]:
endpoint_response = response.json()
pprint.pp(endpoint_response)

Extrahieren der eigentlichen Antwort aus der Rückantwort des Endpunkt

In [ ]:
chat_answer = endpoint_response.get('choices', [])[0].get('message', {}).get('content', "FEHLER")
chat_answer

# Bilder schicken und Fragen zum Bild beantworten

In [ ]:
import base64
from IPython.display import Image, display

with open("einstein.png", mode='rb') as h:
    image_data = h.read()

display(Image(data=image_data))

In [ ]:
base64_encoded = base64.b64encode(image_data).decode('utf-8')

image_payload = {
  # Das 4o klappt in der Regel besser für Bildanalysen
  "model": "gpt-4o",
  "messages": [
    {"role": "system", 
     "content": "Du bist ein Experte darin Fotos zu analysieren und Detailfragen zu beantworten"},
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": """Beantworte mir die folgenden Fragen: 
                      a) Lehnt die Person gegen einen Baum? 
                      b) Trägt die Person einen Hut? 
                      c) Schaut die Person in die Kamera? 
                      d) Hat die Person einen Ziegenbart? 
                      e) Hat die Person einen Schnurrbart?"""
        },
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/png;base64,{base64_encoded}"
          }
        }         
      ]
    }
  ],
  "temperature": 0.5
}

In [ ]:
import pprint
import requests
ENDPOINT = "https://api.openai.com/v1/chat/completions"
    
response = requests.post(ENDPOINT, headers=headers, json=image_payload)
if response.status_code != 200:
    logger.warning(f"Fehlerhafte Antwort: {response.text}")
endpoint_response = response.json()

In [ ]:
chat_answer = endpoint_response.get('choices', [])[0].get('message', {}).get('content', "FEHLER")
chat_answer

# Werte Extraktion mit Schema Aufrufen

In [ ]:
# Quelle: https://de.wikipedia.org/wiki/Albert_Einstein
text = """
       Albert Einstein (* 14. März 1879 in Ulm; † 18. April 1955 in Princeton, New Jersey) war ein theoretischer Physiker. Er gilt als einer der bedeutendsten Physiker der Wissenschaftsgeschichte und weltweit als einer der bekanntesten Wissenschaftler der Neuzeit. Seine Forschungen zur Struktur von Materie, Raum und Zeit sowie zum Wesen der Gravitation veränderten maßgeblich das zuvor geltende newtonsche Weltbild. 1999 wurde Albert Einstein in einer durch die Fachzeitschrift Physics World durchgeführten Umfrage unter führenden Physikern vor Isaac Newton, James Clerk Maxwell, Niels Bohr und Werner Heisenberg zum bedeutendsten Physiker aller Zeiten gewählt.[1]
       Einsteins Hauptwerk, die Relativitätstheorie, machte ihn weltberühmt. Im Jahr 1905 erschien seine Arbeit mit dem Titel Zur Elektrodynamik bewegter Körper, deren Inhalt heute als Spezielle Relativitätstheorie bezeichnet wird. 1915 publizierte er die Allgemeine Relativitätstheorie. Auch zur Quantenphysik leistete er wesentliche Beiträge. „Für seine Verdienste um die Theoretische Physik, besonders für seine Entdeckung des Gesetzes des photoelektrischen Effekts“, erhielt er den Nobelpreis des Jahres 1921, der ihm 1922 überreicht wurde. Seine theoretischen Arbeiten spielten – im Gegensatz zur weit verbreiteten Meinung – beim Bau der Atombombe und der Entwicklung der Kernenergie nur eine indirekte Rolle.[2]

        1909: Ehrendoktor der Universität Genf.[157]
        1914: Ordentliches Mitglied der Preußischen Akademie der Wissenschaften. 1933 erklärte Einstein seinen Austritt aus der Akademie.[158]
        1915: Korrespondierendes Mitglied der Akademie der Wissenschaften zu Göttingen (ab 1923 auswärtiges Mitglied).[159] 1933 wurde Einstein ausgeschlossen.[160]
        1917: Ehrenpreis der Peter-Wilhelm-Müller-Stiftung in der Kategorie Mathematik, gemeinsam mit David Hilbert.
        1919: Am 12. November wurde Einstein zum 500-jährigen Bestehen der Universität Rostock die Ehrendoktorwürde der Medizin (Dr. med. h.c.) verliehen auf Anregung von Moritz Schlick;[161] dies war die einzige deutsche Ehrendoktorwürde, die Einstein erhalten hat.[162]
        1920: Mitglied der Königlich Dänischen Akademie der Wissenschaften und der Königlich Niederländischen Akademie der Wissenschaften.[157]
        1920: Wahl in den Orden Pour le Mérite als dessen jüngstes Mitglied. Einstein gab bei seiner Emigration 1933 sein Ordenszeichen an den Ordenskanzler Max Planck (1858–1947) zurück; einen Wiedereintritt lehnte er[163] 1951 ab.
        1921: Barnard-Medaille.
        1921: Ehrendoktor der University of Manchester.[157]       
       """

In [ ]:
schema = {
    "type": "function",
    "function": {
        "name": "extraction_key_information",
        "description": "Extrahiere Schlüsselinformationen über Personen aus Beschreibungstexten",
        "parameters": {
            "type": "object",
            "properties": {   
                "geburtsort": {
                    "type": "string",  
                    "description": "Den Geburtsort der Person",
                },
                "auszeichnungen": {
                    "type": "array",  
                    "description": "Die Liste der Auszeichnung von Albert Einstein",
                    "items": {
                        "type": "object",
                        "properties": {
                            "jahr": {
                                "type": "number",
                                "description": "Das Jahr der Auszeichnung"
                            },
                            "beschreibungstext": {
                                "type": "string",
                                "description": "Beschreibung der Auszeichnung als ein Satz"
                            },                            
                        }
                    }
                }                
            }
        }
    }
}

In [ ]:
payload = {
  "model": "gpt-4o-mini",
  "messages": [
    {
        "role": "system", 
        "content": "Du bist ein Analytiker, der Informationen aus Textblöcken extrahiert."
    },
    {
      "role": "user",
      "content": text
    }
  ],
  "temperature": 0.5,
  "tools": [schema],
  "tool_choice": "required",    
}

In [ ]:
ENDPOINT = "https://api.openai.com/v1/chat/completions"
    
response = requests.post(ENDPOINT, headers=headers, json=payload)
if response.status_code != 200:
    logger.warning(f"Fehlerhafte Antwort: {response.text}")
endpoint_response = response.json()

In [ ]:
endpoint_response = response.json()
pprint.pp(endpoint_response)

In [ ]:
import json
# Sobald wird ein tooling Aufruf benutzen, ändert sich die Datenstruktur, die wir zurückkriegen
for choice in endpoint_response.get('choices', []):
    message = choice.get('message', {})
    for tool_call_output in message.get('tool_calls', {}):
        result_jsonized = tool_call_output.get('function', {}).get('arguments')
        for key, value in json.loads(result_jsonized).items():
            if isinstance(value, str):
                print(key, ": ", value)
            else:
                for nested_dict in value:
                    for _k, _v in nested_dict.items():
                        print(_k, ": ", _v)
        print("======")